In [1]:
from IPython import get_ipython
from IPython.core.display import display
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
get_ipython().run_line_magic('run', 'setup')

bevolking = CBS.bevolking().iloc[0]
#display(bevolking.BevolkingOpDeEersteVanDeMaand)
#display(bevolking['per 100k'])

def addstats(df):
  global dagoverzicht

  # de aanname is dat df al gegroepeerd is op datum, en dat de kolommen de dagtotalen zijn
  dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)
  for stat in df.columns:
    # nieuw = dagtotaal, zet missende dagtotalen op 0
    dagoverzicht[f'{stat} (nieuw)'] = dagoverzicht[stat].fillna(0).astype(int)
    # kolom met eerder de dagtotalen = cumulatieve som over dagtotalen
    dagoverzicht[stat] = dagoverzicht[f'{stat} (nieuw)'].cumsum()
    # factor voor 100k
    dagoverzicht[f'{stat} per 100.000'] = dagoverzicht[stat] * bevolking['per 100k']
    # verschil met 7 dagen terug
    dagoverzicht[f'{stat} 7d'] = dagoverzicht[f'{stat} (nieuw)'].rolling(7).sum().fillna(0)
    # en weer factor 100 k
    dagoverzicht[f'{stat} 7d per 100.000'] = dagoverzicht[f'{stat} 7d'] * bevolking['per 100k']

In [2]:
@run('set up base frame from ESRI -> NICE')
def cell():
  df = ArcGIS.nice('f27f743476a142538e8054f7a7ce12e1')

  df['date'] = pd.to_datetime(df.date.str.replace(' .*', '', regex=True))
  df.set_index('date', inplace=True)

  # base date range uit NICE
  global dagoverzicht
  # maak leeg dataframe met een rij voor elke dag van 2020-02-27 tm Date_of_publication
  dagoverzicht = pd.DataFrame(index=pd.date_range(start='2020-02-27', end=df.index.max()))
  # noem de index Key
  dagoverzicht.index.name='Key'
  # vul de datum kolom
  dagoverzicht['Datum'] = dagoverzicht.index.strftime('%Y-%m-%d')
  # vaste waarde voor LandCode
  dagoverzicht['LandCode'] = 'NL'

  # the NICE data as prepared by ESRI can just be used as-is, so just rename the columns
  for prefix, kind in [ ('ic', 'IC'), ('zkh', 'Ziekenhuis') ]:
    df = df.rename(columns={
      f'{prefix}NewIntake': f'NICE {kind} Bedden (intake)',
      f'{prefix}IntakeCount': f'NICE {kind} Bedden',
      f'{prefix}IntakeCumulative': f'NICE {kind} Bedden (cumulatief)',
      f'{prefix}DiedCumulative': f'NICE {kind} Overleden',
    })

  # remove the columns we don't use
  df = df[[col for col in df.columns if 'NICE' in col]]
  dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)

  # after the merge, dates missing in the ESRI/NICE set will be `nan`, so fill these
  for col in df.columns:
    # for cumulatief columns, fill-forward from last known value
    if 'cumulatief' in col:
      dagoverzicht[col] = dagoverzicht[col].ffill()
    # for non-cumulative and whatever remains in cumulative after fill forward (which will be leading nan's), fill with 0
    dagoverzicht[col] = dagoverzicht[col].fillna(0)

  for kind in [ 'IC', 'Ziekenhuis' ]:
    dagoverzicht[f'NICE {kind} Bedden (toename)'] = (dagoverzicht[f'NICE {kind} Bedden'] - dagoverzicht[f'NICE {kind} Bedden'].shift(1)).fillna(0)

  for window, shift, past in [(3, 1, 'afgelopen '), (7, 0, '')]:
    dagoverzicht[f'NICE IC Bedden (intake) {past}{window}d'] = dagoverzicht['NICE IC Bedden (intake)'].shift(shift).rolling(window).sum().fillna(0)

  dagoverzicht['NICE IC Bedden (intake) week-1'] = dagoverzicht['NICE IC Bedden (intake) 7d'].shift(7).fillna(0)

set up base frame from ESRI -> NICE


downloading nice/f27f743476a142538e8054f7a7ce12e1_0-2021-02-17@11-21.csv


loading nice/f27f743476a142538e8054f7a7ce12e1_0-2021-02-17@11-21.csv.gz


In [3]:
@run('overleden + positief getest')
def cell():
  df = RIVM.csv('COVID-19_aantallen_gemeente_per_dag').rename(columns={
    'Total_reported': 'Positief getest',
    'Deceased': 'Overleden',
    'Date_of_publication': 'Datum',
    'Date_of_report': 'Today',
  })
  # sloop tijd van de datum en zet om in datetime object
  df['Datum'] = pd.to_datetime(df.Datum.str.replace(' .*', '', regex=True))

  # sommeer pos en overl op datum en voeg toe aan dagoverzicht
  addstats(df.groupby(['Datum']).agg({'Positief getest': 'sum', 'Overleden': 'sum'}))
  global dagoverzicht
  for col in ['Overleden', 'Positief getest']:
    dagoverzicht[f'{col} week-1'] = dagoverzicht[f'{col} 7d'].shift(7).fillna(0)
  display(dagoverzicht.head(10))

overleden + positief getest


rivm/COVID-19_aantallen_gemeente_per_dag-2021-02-16@14-15.csv exists
loading rivm/COVID-19_aantallen_gemeente_per_dag-2021-02-16@14-15.csv.gz


,Datum,LandCode,NICE IC Bedden (intake),NICE IC Bedden (cumulatief),NICE IC Bedden,NICE IC Overleden,NICE Ziekenhuis Bedden (intake),NICE Ziekenhuis Bedden (cumulatief),NICE Ziekenhuis Bedden,NICE Ziekenhuis Overleden,...,Positief getest (nieuw),Positief getest per 100.000,Positief getest 7d,Positief getest 7d per 100.000,Overleden (nieuw),Overleden per 100.000,Overleden 7d,Overleden 7d per 100.000,Overleden week-1,Positief getest week-1
Key,,,,,,,,,,,,,,,,,,,,,
2020-02-27,2020-02-27,NL,1,13,13,0,2,65,62,0,...,0,0.000000,0.0,0.000000,0,0.000000,0.0,0.000000,0.0,0.0
2020-02-28,2020-02-28,NL,0,14,14,0,3,68,65,0,...,1,0.005723,0.0,0.000000,0,0.000000,0.0,0.000000,0.0,0.0
2020-02-29,2020-02-29,NL,0,15,15,0,1,69,65,0,...,1,0.011445,0.0,0.000000,0,0.000000,0.0,0.000000,0.0,0.0
2020-03-01,2020-03-01,NL,2,17,17,0,7,77,73,0,...,3,0.028613,0.0,0.000000,0,0.000000,0.0,0.000000,0.0,0.0
2020-03-02,2020-03-02,NL,0,17,16,0,10,88,84,0,...,5,0.057226,0.0,0.000000,0,0.000000,0.0,0.000000,0.0,0.0
2020-03-03,2020-03-03,NL,1,19,18,0,8,96,91,0,...,4,0.080116,0.0,0.000000,0,0.000000,0.0,0.000000,0.0,0.0
2020-03-04,2020-03-04,NL,3,22,21,0,10,109,103,0,...,10,0.137341,24.0,0.137341,0,0.000000,0.0,0.000000,0.0,0.0
2020-03-05,2020-03-05,NL,1,24,21,0,12,121,113,0,...,17,0.234625,41.0,0.234625,0,0.000000,0.0,0.000000,0.0,0.0
2020-03-06,2020-03-06,NL,4,28,25,0,19,138,129,0,...,40,0.463527,80.0,0.457805,0,0.000000,0.0,0.000000,0.0,0.0


In [4]:
@run('ziekenhuisopnames')
def cell():
  df = RIVM.csv('COVID-19_ziekenhuisopnames').rename(columns={
    'Hospital_admission': 'Ziekenhuisopnames',
    'Date_of_statistics': 'Datum',
  })
  # datum naar datetime
  df['Datum'] = pd.to_datetime(df.Datum.str.replace(' .*', '', regex=True))
  # sommeer op datum en voeg toe aan dagoverzicht
  addstats(df.groupby(['Datum']).agg({'Ziekenhuisopnames': 'sum'}))
  dagoverzicht['Ziekenhuisopnames week-1'] = dagoverzicht['Ziekenhuisopnames 7d'].shift(7).fillna(0)
  display(dagoverzicht.head())

ziekenhuisopnames


rivm/COVID-19_ziekenhuisopnames-2021-02-16@14-15.csv exists
loading rivm/COVID-19_ziekenhuisopnames-2021-02-16@14-15.csv.gz


,Datum,LandCode,NICE IC Bedden (intake),NICE IC Bedden (cumulatief),NICE IC Bedden,NICE IC Overleden,NICE Ziekenhuis Bedden (intake),NICE Ziekenhuis Bedden (cumulatief),NICE Ziekenhuis Bedden,NICE Ziekenhuis Overleden,...,Overleden 7d,Overleden 7d per 100.000,Overleden week-1,Positief getest week-1,Ziekenhuisopnames,Ziekenhuisopnames (nieuw),Ziekenhuisopnames per 100.000,Ziekenhuisopnames 7d,Ziekenhuisopnames 7d per 100.000,Ziekenhuisopnames week-1
Key,,,,,,,,,,,,,,,,,,,,,
2020-02-27,2020-02-27,NL,1,13,13,0,2,65,62,0,...,0.0,0.0,0.0,0.0,3,3,0.017168,0.0,0.0,0.0
2020-02-28,2020-02-28,NL,0,14,14,0,3,68,65,0,...,0.0,0.0,0.0,0.0,6,3,0.034335,0.0,0.0,0.0
2020-02-29,2020-02-29,NL,0,15,15,0,1,69,65,0,...,0.0,0.0,0.0,0.0,8,2,0.045780,0.0,0.0,0.0
2020-03-01,2020-03-01,NL,2,17,17,0,7,77,73,0,...,0.0,0.0,0.0,0.0,17,9,0.097284,0.0,0.0,0.0
2020-03-02,2020-03-02,NL,0,17,16,0,10,88,84,0,...,0.0,0.0,0.0,0.0,27,10,0.154509,0.0,0.0,0.0


In [5]:
@run('reproductiegetal en besmettelijkheid')
def cell():
  global dagoverzicht

  datasets = [
    ('COVID-19_reproductiegetal', 'Rt_avg', 'Reproductiegetal'),
    ('COVID-19_prevalentie', 'prev_avg', 'Besmettelijk'),
  ]
  for dataset, source, target in datasets:
    # laad de dataset
    df = RIVM.json(dataset).rename(columns={source: target})
    # Date naar datetime index voor de merge
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    # merge alleen de target kolom
    df = df[[target]]
    # voeg to aan dagoverzicht
    dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)
    # vul lege waarden met 0
    dagoverzicht[target] = dagoverzicht[target].fillna(0)
  # per 100k factor
  dagoverzicht['Besmettelijk per 100.000'] = (dagoverzicht['Besmettelijk']  * bevolking['per 100k']).round(0)
  display(dagoverzicht)

reproductiegetal en besmettelijkheid


rivm/COVID-19_reproductiegetal-2021-02-16@14-15.json exists
loading rivm/COVID-19_reproductiegetal-2021-02-16@14-15.json.gz


rivm/COVID-19_prevalentie-2021-02-16@14-15.json exists
loading rivm/COVID-19_prevalentie-2021-02-16@14-15.json.gz


,Datum,LandCode,NICE IC Bedden (intake),NICE IC Bedden (cumulatief),NICE IC Bedden,NICE IC Overleden,NICE Ziekenhuis Bedden (intake),NICE Ziekenhuis Bedden (cumulatief),NICE Ziekenhuis Bedden,NICE Ziekenhuis Overleden,...,Positief getest week-1,Ziekenhuisopnames,Ziekenhuisopnames (nieuw),Ziekenhuisopnames per 100.000,Ziekenhuisopnames 7d,Ziekenhuisopnames 7d per 100.000,Ziekenhuisopnames week-1,Reproductiegetal,Besmettelijk,Besmettelijk per 100.000
Key,,,,,,,,,,,,,,,,,,,,,
2020-02-27,2020-02-27,NL,1,13,13,0,2,65,62,0,...,0.0,3,3,0.017168,0.0,0.000000,0.0,2.06,6808.0,39.0
2020-02-28,2020-02-28,NL,0,14,14,0,3,68,65,0,...,0.0,6,3,0.034335,0.0,0.000000,0.0,2.10,8500.0,49.0
2020-02-29,2020-02-29,NL,0,15,15,0,1,69,65,0,...,0.0,8,2,0.045780,0.0,0.000000,0.0,1.99,10568.0,60.0
2020-03-01,2020-03-01,NL,2,17,17,0,7,77,73,0,...,0.0,17,9,0.097284,0.0,0.000000,0.0,1.83,13044.0,75.0
2020-03-02,2020-03-02,NL,0,17,16,0,10,88,84,0,...,0.0,27,10,0.154509,0.0,0.000000,0.0,1.79,15996.0,92.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-13,2021-02-13,NL,21,8285,536,2384,107,39047,1511,5677,...,27172.0,43837,107,250.859915,1119.0,6.403546,1208.0,0.00,0.0,0.0
2021-02-14,2021-02-14,NL,23,8311,542,2395,133,39173,1529,5703,...,27414.0,43946,109,251.483674,1120.0,6.409269,1166.0,0.00,0.0,0.0
2021-02-15,2021-02-15,NL,32,8346,555,2401,179,39362,1643,5725,...,26430.0,43981,35,251.683964,967.0,5.533717,1139.0,0.00,0.0,0.0


In [6]:
@run('uitgevoerde testen')
def cell():
  df = RIVM.csv('COVID-19_uitgevoerde_testen').rename(columns={
    'Date_of_statistics': 'Datum',
    'Tested_with_result': 'GGD getest',
    'Tested_positive': 'GGD getest positief',
  })
  df['Datum'] = pd.to_datetime(df.Datum)
  df = df.groupby(['Datum']).agg({'GGD getest': 'sum', 'GGD getest positief': 'sum'})
  display(df)

  global dagoverzicht
  columns = dagoverzicht.columns

  dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)

  dagoverzicht['GGD percentage positief'] = (dagoverzicht['GGD getest positief'] / dagoverzicht['GGD getest']).fillna(0)

  dagoverzicht['GGD getest (7 daags)'] = dagoverzicht['GGD getest'].rolling(7).sum().fillna(0)
  dagoverzicht['GGD getest positief (7 daags)'] = dagoverzicht['GGD getest positief'].rolling(7).sum().fillna(0)

  dagoverzicht['GGD percentage positief (7 daags)'] = (dagoverzicht['GGD getest positief (7 daags)'] / dagoverzicht['GGD getest (7 daags)']).fillna(0)

  dagoverzicht['GGD getest (cumulatief)'] = dagoverzicht['GGD getest'].cumsum()
  dagoverzicht['GGD getest positief (cumulatief)'] = dagoverzicht['GGD getest positief'].cumsum()

  dagoverzicht['GGD percentage positief (cumulatief)'] = (dagoverzicht['GGD getest positief (cumulatief)'] / dagoverzicht['GGD getest (cumulatief)']).fillna(0)

  # fill 0 *after* the calculations above to prevent summing 'GGD getest (7 daags) from the starting date of the 'GGD getest' series
  for col in dagoverzicht.columns:
    # only zero out new columns
    if col in columns: continue
    dagoverzicht[col] = dagoverzicht[col].fillna(0)

  display(dagoverzicht.head(10))

uitgevoerde testen


rivm/COVID-19_uitgevoerde_testen-2021-02-16@14-15.csv exists
loading rivm/COVID-19_uitgevoerde_testen-2021-02-16@14-15.csv.gz


,GGD getest,GGD getest positief
Datum,,
2020-06-01,1552,73
2020-06-02,6819,203
2020-06-03,8867,165
2020-06-04,9339,171
2020-06-05,9464,135
...,...,...
2021-02-10,32407,3635
2021-02-11,31100,3271
2021-02-12,27650,2935


,Datum,LandCode,NICE IC Bedden (intake),NICE IC Bedden (cumulatief),NICE IC Bedden,NICE IC Overleden,NICE Ziekenhuis Bedden (intake),NICE Ziekenhuis Bedden (cumulatief),NICE Ziekenhuis Bedden,NICE Ziekenhuis Overleden,...,Besmettelijk per 100.000,GGD getest,GGD getest positief,GGD percentage positief,GGD getest (7 daags),GGD getest positief (7 daags),GGD percentage positief (7 daags),GGD getest (cumulatief),GGD getest positief (cumulatief),GGD percentage positief (cumulatief)
Key,,,,,,,,,,,,,,,,,,,,,
2020-02-27,2020-02-27,NL,1,13,13,0,2,65,62,0,...,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-28,2020-02-28,NL,0,14,14,0,3,68,65,0,...,49.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-29,2020-02-29,NL,0,15,15,0,1,69,65,0,...,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-01,2020-03-01,NL,2,17,17,0,7,77,73,0,...,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-02,2020-03-02,NL,0,17,16,0,10,88,84,0,...,92.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-03,2020-03-03,NL,1,19,18,0,8,96,91,0,...,111.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-04,2020-03-04,NL,3,22,21,0,10,109,103,0,...,133.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-05,2020-03-05,NL,1,24,21,0,12,121,113,0,...,158.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-06,2020-03-06,NL,4,28,25,0,19,138,129,0,...,186.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
@run('LCPS')
def cell():
  # laad dataset
  df = LCPS.csv('covid-19').rename(columns={
    'IC_Bedden_COVID': 'LCPS IC Bedden COVID',
    'IC_Bedden_Non_COVID': 'LCPS IC Bedden Non COVID',
    'Kliniek_Bedden': 'LCPS Kliniek Bedden COVID',
    'IC_Nieuwe_Opnames_COVID': 'LCPS IC Nieuwe Opnames COVID',
    'Kliniek_Nieuwe_Opnames_COVID': 'LCPS Kliniek Nieuwe Opnames COVID',
  })
  # datum naar datetime index voor merge
  df['Datum'] = pd.to_datetime(df['Datum'], format='%d-%m-%Y')
  df.set_index('Datum', inplace=True)

  # sommeer op datum
  df = df.groupby(['Datum']).agg({col: 'sum' for col in df.columns})

  # toename = waarde - vorige
  df['LCPS IC Bedden COVID (toename)'] = (df['LCPS IC Bedden COVID'] - df['LCPS IC Bedden COVID'].shift(1)).fillna(0)
  df['LCPS Kliniek Bedden COVID (toename)'] = (df['LCPS Kliniek Bedden COVID'] - df['LCPS Kliniek Bedden COVID'].shift(1)).fillna(0)

  global dagoverzicht
  # voeg toe aan dagoverzicht
  dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)
  for col in df.columns:
    # vervang lege waarden door 0
    dagoverzicht[col] = dagoverzicht[col].fillna(0).astype(int)

LCPS


downloading lcps/covid-19-2021-02-17@13-00.csv


loading lcps/covid-19-2021-02-17@13-00.csv.gz


In [8]:
@run('corrections')
def cell():
  # laad corrections van mzelst
  df = GitHub.csv('mzelst/covid-19/contents/corrections/corrections_perday.csv')
  # date naar datetime index voor merge
  df['date'] = pd.to_datetime(df.date)
  df.set_index('date', inplace=True)
  columns =  {
    'net.infection': 'Positief getest (toename)',
    'net.hospitals': 'Ziekenhuisopnames (toename)',
    'net.deaths': 'Overleden (toename)'
  }
  # rename kolommen naar onze namen
  df = df.rename(columns=columns)[list(columns.values())]

  global dagoverzicht
  # voeg toe aan dagoverxicht
  dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)
  for col in columns.values():
    # set lege waarden op 0
    dagoverzicht[col] = dagoverzicht[col].fillna(0).astype(int)

corrections
https://api.github.com/repos/mzelst/covid-19/contents/corrections/corrections_perday.csv
downloading github/corrections_perday-2021-02-17@12-24.csv


In [9]:
@run('Personen')
def cell():
  global dagoverzicht
  dagoverzicht['Personen'] = bevolking.BevolkingOpDeEersteVanDeMaand
  display(dagoverzicht)

Personen


,Datum,LandCode,NICE IC Bedden (intake),NICE IC Bedden (cumulatief),NICE IC Bedden,NICE IC Overleden,NICE Ziekenhuis Bedden (intake),NICE Ziekenhuis Bedden (cumulatief),NICE Ziekenhuis Bedden,NICE Ziekenhuis Overleden,...,LCPS IC Bedden Non COVID,LCPS Kliniek Bedden COVID,LCPS IC Nieuwe Opnames COVID,LCPS Kliniek Nieuwe Opnames COVID,LCPS IC Bedden COVID (toename),LCPS Kliniek Bedden COVID (toename),Positief getest (toename),Ziekenhuisopnames (toename),Overleden (toename),Personen
Key,,,,,,,,,,,,,,,,,,,,,
2020-02-27,2020-02-27,NL,1,13,13,0,2,65,62,0,...,0,0,0,0,0,0,0,0,0,17474693
2020-02-28,2020-02-28,NL,0,14,14,0,3,68,65,0,...,0,0,0,0,0,0,0,0,0,17474693
2020-02-29,2020-02-29,NL,0,15,15,0,1,69,65,0,...,0,0,0,0,0,0,0,0,0,17474693
2020-03-01,2020-03-01,NL,2,17,17,0,7,77,73,0,...,0,0,0,0,0,0,0,0,0,17474693
2020-03-02,2020-03-02,NL,0,17,16,0,10,88,84,0,...,0,0,0,0,0,0,0,0,0,17474693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-13,2021-02-13,NL,21,8285,536,2384,107,39047,1511,5677,...,522,1325,32,182,4,-45,4194,61,66,17474693
2021-02-14,2021-02-14,NL,23,8311,542,2395,133,39173,1529,5703,...,470,1329,19,135,-8,4,3447,61,23,17474693
2021-02-15,2021-02-15,NL,32,8346,555,2401,179,39362,1643,5725,...,499,1386,22,143,6,57,2810,45,27,17474693


In [10]:
async def publish():
  global dagoverzicht

  m = (dagoverzicht == np.inf)
  df = dagoverzicht.loc[m.any(axis=1), m.any(axis=0)]
  display(df.head())
  m = (dagoverzicht == np.nan)
  df = dagoverzicht.loc[m.any(axis=1), m.any(axis=0)]
  display(df.head())

  os.makedirs('artifacts', exist_ok = True)
  dagoverzicht.to_csv('artifacts/DagOverzicht.csv', index=True)

  if knack:
    print('updating knack')
    df = dagoverzicht.assign(Key=dagoverzicht.index.strftime('%Y-%m-%d'))
    await knack.update(objectName='Dagoverzicht', df=df)
    await knack.timestamps('Dagoverzicht', Cache.timestamps)
await publish()

""
Key


""
Key


updating knack


  0%|          | 0/330 [00:00<?, ?it/s]

  0%|          | 1/330 [00:00<04:18,  1.27it/s]

...


rate limit: 7 
API calls: create: 0, read: 2, update: 330, delete: 0, backoff: 7
errors:
  503: Service Unavailable: 4


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  1.26it/s]

100%|██████████| 1/1 [00:00<00:00,  1.26it/s]


rate limit: 7 
API calls: create: 0, read: 1, update: 1, delete: 0, backoff: 0
